In [1]:
import pandas as pd
import numpy as np
import joblib

Загружаем необходимые данные

In [2]:
length_of_sequence, clf = joblib.load('model_and_size.joblib')

In [3]:
length_of_sequence

76.0

Считываем тестовый датасет

In [4]:
df = pd.read_parquet('test.parquet')

In [5]:
df.head()

,id,dates,values
0,6125,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[1.85, -0.04, 0.19, -0.45, -0.75, -0.95, -2.91..."
1,26781,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-0.41, 0.39, -0.47, -0.9, -1.46, -0.51, 0.51,..."
2,13333,"[2016-06-01, 2016-07-01, 2016-08-01, 2016-09-0...","[-0.29, -1.26, 0.17, -1.22, 0.45, -0.94, 0.16,..."
3,53218,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[-1.47, 1.55, -0.03, 0.57, -0.57, 0.6, 0.27, 1..."
4,84204,"[2016-01-01, 2016-02-01, 2016-03-01, 2016-04-0...","[2.33, 1.39, -1.03, -2.64, 1.89, 1.77, 1.43, 1..."


Генерируем те же признаки, что и для обучения

In [6]:
def process(dF, length_of_sequence):

    df = pd.DataFrame()

    df['length'] = dF['dates'].apply(len)
    df['mean'] = dF['values'].apply(np.mean)
    df['std'] = dF['values'].apply(np.std)
    df['max'] = dF['values'].apply(np.max)
    df['min'] = dF['values'].apply(np.min)
    df['median'] = dF['values'].apply(np.median)
    df['trend'] = dF['values'].apply(lambda x: x[-1] - x[0])

    

    perfect_len = length_of_sequence
    def perfectify_list(array = np.array):
        if len(array) > perfect_len:
            diff = (len(array) - perfect_len)
            # print(diff, '>')
            perfect_array = array[int(diff):]
            return perfect_array
        if len(array) < perfect_len:
            diff = - (len(array) - perfect_len)
            # print(diff, '<')
            perfect_array = np.concatenate((np.array([array[0]] * int(diff)), array))
            return perfect_array
        return array

    # df['perfect_values'] = dF['values'].apply(perfectify_list)
    df['start_month'] = dF['dates'].apply(lambda x: x[0].month)
    df['start_year'] = dF['dates'].apply(lambda x: x[0].year)

    df['end_month'] = dF['dates'].apply(lambda x: x[-1].month)
    df['end_year'] = dF['dates'].apply(lambda x: x[-1].year)

    values_columns = pd.DataFrame(dF['values'].apply(perfectify_list).tolist(), index=df.index)
    values_columns.columns = [f'value_{i}' for i in range(values_columns.shape[1])]

    df = pd.concat([dF['id'], values_columns, df], axis=1)

    return df


In [7]:
processed_df = process(df, length_of_sequence)

Итоговый тестовый датасет для предсказания

In [8]:
processed_df.head()

,id,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,...,mean,std,max,min,median,trend,start_month,start_year,end_month,end_year
0,6125,1.85,1.85,1.85,1.85,1.85,1.85,1.85,1.85,1.85,...,0.037895,1.505985,3.13,-3.14,-0.01,-2.120000,1,2016,9,2020
1,26781,-0.41,-0.41,-0.41,-0.41,-0.41,-0.41,-0.41,-0.41,-0.41,...,-0.016909,1.249663,3.85,-2.24,-0.08,2.790000,1,2016,7,2020
2,13333,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,-0.29,...,-0.039569,1.000514,3.46,-1.45,-0.16,1.677809,6,2016,12,2020
3,53218,-0.03,0.57,-0.57,0.60,0.27,1.29,0.73,0.05,0.26,...,0.159231,1.313112,3.43,-2.73,0.11,0.790000,1,2016,6,2022
4,84204,2.33,2.33,2.33,2.33,2.33,2.33,2.33,2.33,2.33,...,-0.094705,1.243279,2.57,-2.66,-0.09,-3.827317,1,2016,5,2021


Предсказываем вероятности принадлежности

In [9]:
y_predict_proba = clf.predict_proba(processed_df.drop('id', axis = 1))

In [10]:
y_predict_proba

array([[0.96148026, 0.03851971],
       [0.7374753 , 0.2625247 ],
       [0.60784006, 0.3921599 ],
       ...,
       [0.98210573, 0.01789425],
       [0.6182391 , 0.3817609 ],
       [0.9900022 , 0.0099978 ]], dtype=float32)

Берем второй столбец, т.к. это вероятность принадлежности к 1 классу

In [11]:
y_predict_final = pd.DataFrame([prob[1] for prob in y_predict_proba], columns = [['score']])

In [12]:
y_predict_final

,score
0,0.038520
1,0.262525
2,0.392160
3,0.027306
4,0.565654
...,...
19995,0.321533
19996,0.037884
19997,0.017894
19998,0.381761


Объединяем со столбцом индексов

In [13]:
final_df = pd.DataFrame()
final_df['id'] = processed_df['id']
final_df['score'] = y_predict_final['score']

In [14]:
final_df

,id,score
0,6125,0.038520
1,26781,0.262525
2,13333,0.392160
3,53218,0.027306
4,84204,0.565654
...,...,...
19995,80341,0.321533
19996,5891,0.037884
19997,29091,0.017894
19998,85877,0.381761


Записываем в файл submission.csv

In [15]:
final_df.to_csv('submission.csv', sep=';', encoding='utf-8', index=False, header=True)